# Исследование надежности заемщиков

## 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('D:/Downloads/data.csv')
display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Предобработка данных

### 2.1 Заполнение пропусков

In [3]:
data.groupby('income_type')['days_employed', 'total_income', 'purpose'].count()

C:\Users\0E3B~1\AppData\Local\Temp/ipykernel_18416/655079050.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data.groupby('income_type')['days_employed', 'total_income', 'purpose'].count()


,days_employed,total_income,purpose
income_type,,,
безработный,2,2,2
в декрете,1,1,1
госслужащий,1312,1312,1459
компаньон,4577,4577,5085
пенсионер,3443,3443,3856
предприниматель,1,1,2
сотрудник,10014,10014,11119
студент,1,1,1


<div style="border:solid blue 1px; padding: 20px"> 
В разбивке по типу занятности не хватает данных по всем целевым категориям (т.е. кроме берзработных, в декрете и студентов). Скорее всего допущена техническая ошибка, или же данные не были предоставлены клиентами.

In [4]:
pass_ = data.isna().sum() / len(data)

In [5]:
print(pass_)

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64


<div style="border:solid blue 1px; padding: 20px"> 
Доля пропусков в колонках days_employed и total_income составила 10,09%.
Значения в данных колонках являются количественными. Заменим их медианными значениями.
Медиана в отличие от среднего арифметического даст более точное значение, так как значения в данных столбцах сильно разнятся.

In [6]:
days_employed_median = data['days_employed'].median()
total_income_median = data['total_income'].median()

data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div style="border:solid blue 1px; padding: 20px"> 
Пропуски заполнены

### 2.2 Проверка данных на аномалии и исправления

In [8]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<div style="border:solid blue 1px; padding: 20px"> 
Проверим данные в колонках с категориальными переменными.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

<div style="border:solid blue 1px; padding: 20px"> 
При записи данных о количесте детей также произошла ошибка: некоторые цифры отражены в отрицательном виде. Приведем их к значению по модулю.

In [10]:
data['children'] = abs(data['children'])
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5], dtype=int64)

In [11]:
data['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [12]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [13]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [14]:
data['gender'].unique()
data[data['gender']=='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


<div style="border:solid blue 1px; padding: 20px"> 
Обнаружена 1 строка с пропущушенной информацией по полу клиента. По остальным столбцам информация есть. В данном исследовании пол клиента - не ключевая информация. Можем оставить эту строку.

In [15]:
data['debt'].unique()

array([0, 1], dtype=int64)

<div style="border:solid blue 1px; padding: 20px"> 
Далее проверим колонки с количественными переменными.
При записи данных о трудовом стаже произошла ошибка: некоторые цифры отражены в отрицательном виде. Приведем их к значению по модулю, а также округлим до целого числа.

In [16]:
data['days_employed'] = abs(data['days_employed']).astype(int)

In [17]:
data['days_employed'].max()

401755

<div style="border:solid blue 1px; padding: 20px"> 
При записи данных о трудовом стаже есть еще ошибки. Максимальное значение в столбце - 401755 дней (или 1100 лет).

In [18]:
data['days_employed'].sort_values(ascending=False).head(3448)

6954     401755
10006    401715
7664     401675
2156     401674
7794     401663
          ...  
9328     328734
20444    328728
16335     18388
4299      17615
7329      16593
Name: days_employed, Length: 3448, dtype: int32

In [19]:
data[data['days_employed']>300000].groupby('dob_years')['days_employed'].count()

dob_years
0      17
22      1
26      2
27      3
28      1
31      1
32      3
33      2
34      3
35      1
36      3
37      5
38      8
39      4
40      7
41      6
42      9
43      9
44     10
45     11
46     13
47     13
48     20
49     30
50     61
51     73
52     95
53    105
54    145
55    162
56    184
57    212
58    208
59    254
60    243
61    214
62    235
63    192
64    179
65    136
66    139
67    132
68     80
69     74
70     54
71     48
72     28
73      6
74      4
Name: days_employed, dtype: int64

<div style="border:solid blue 1px; padding: 20px"> 
Даже если предположить, что 401755 - это количество отработанных часов, то все равно 45 лет стажа выглядит нереально, учитывая, что в нашей выборке есть лица, не достигшие такого возраста.
Заменим все значения больше 300000 медианным значением.

In [20]:
def replace_wrong_values(wrong_values, correct_value):
    for wrong_value in wrong_values:
        data['days_employed'] = data['days_employed'].replace(wrong_value, correct_value)

wrong_values = range(300000,402000)
correct_value = days_employed_median
replace_wrong_values(wrong_values, correct_value)
data['days_employed'].sort_values(ascending=False).head()

16335    18388.0
4299     17615.0
7329     16593.0
17838    16264.0
16825    16119.0
Name: days_employed, dtype: float64

In [21]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

<div style="border:solid blue 1px; padding: 20px"> 
В данных некоторых пользователей не указан возраст. Заменим нули медианным значением.

In [22]:
dob_years_median = data['dob_years'].median()
data['dob_years'] = data['dob_years'].replace(0,dob_years_median)
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

### 2.3 Изменение типов данных

<div style="border:solid blue 1px; padding: 20px"> 
Также округлим до целого числа значения в колонке total_income.

In [23]:
data['total_income'] = data['total_income'].astype(int)

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


### 2.4 Удаление дубликатов

<div style="border:solid blue 1px; padding: 20px"> 
Проверим данные на неявные дубликаты.

In [25]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [26]:
data['education'] = data['education'].str.lower()

In [27]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [28]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [29]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

<div style="border:solid blue 1px; padding: 20px"> 
Неявные дубликаты в столбце education удалены. В столбце purpose очень много похожих значений. Далее сгрупируем их в несколько укрупненных категорий.

<div style="border:solid blue 1px; padding: 20px"> 
Проверим данные на явные дубликаты.

In [30]:
data.duplicated().sum()

72

<div style="border:solid blue 1px; padding: 20px"> 
Всего в данных 72 явных дубликата

In [31]:
data = data.drop_duplicates().reset_index(drop=True)

In [32]:
data.duplicated().sum()

0

### 2.5 Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [33]:
education_dict = data[['education_id', 'education']]
family_status_dict = data[['family_status_id', 'family_status']]

<div style="border:solid blue 1px; padding: 20px"> 
Удалим дубликаты

In [34]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

In [35]:
data = data.drop('education', 1)
data = data.drop('family_status', 1)

C:\Users\0E3B~1\AppData\Local\Temp/ipykernel_18416/434605108.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('education', 1)
C:\Users\0E3B~1\AppData\Local\Temp/ipykernel_18416/434605108.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('family_status', 1)


### 2.6 Категоризация дохода

<div style="border:solid blue 1px; padding: 20px"> 
Создадим категории по доходам клиентов

In [36]:
def income_category (row):
    income = row['total_income']
    if income < 30001:
        return 'E'
    if income < 50001:
        return 'D'
    if income < 200001:
        return 'C'
    if income < 1000001:
        return 'B'
    else:
        return 'A'
data['total_income_category'] = data.apply(income_category, axis=1)

In [37]:
data['total_income_category'].value_counts()

C    16015
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### 2.7 Категоризация целей кредита

<div style="border:solid blue 1px; padding: 20px"> 
Создадим категории по целям кредита

In [38]:
def purpose_category (row):
    purpose = row['purpose']
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'автомоб' in purpose:
        return 'операции с автомобилем'
    if 'образ' in purpose:
        return 'получение образования'
    if 'жиль' or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    else:
        return 'другое'
data['purpose_category'] = data.apply(purpose_category, axis=1)      

In [39]:
def purpose_category_2 (row):
    purpose = row['children']
    if purpose == 0:
        return '0'
    if purpose < 3:
        return '1-2'
    else:
        return '3+'
data['purpose_category_2'] = data.apply(purpose_category_2, axis=1)  

In [40]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,purpose_category_2
0,1,8437.000000,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1-2
1,1,4024.000000,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1-2
2,0,5623.000000,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,0
3,3,4124.000000,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,3+
4,0,-1203.369529,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,0


## 3. Ответы на вопросы

### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [41]:
round(data.pivot_table(index='purpose_category_2', values='debt', aggfunc='mean'),3)

,debt
purpose_category_2,
0,0.075
1-2,0.093
3+,0.086


### Вывод 1:
Доли по всем категориям примерно одинаковые. Около 90% клиентов не имели проблем с возвратом кредита.
Быстрее выплачивают кредиты семьи без детей (у 92,5% не было задолженностей). Хуже всего возвращают кредиты с 1-2 детьми(9,3% имели задолженность по кредиту).

### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
round(data.pivot_table(index='family_status_id', values='debt', aggfunc='mean'),3)

,debt
family_status_id,
0,0.075
1,0.093
2,0.066
3,0.071
4,0.098


In [43]:
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Вывод 2:

Доли по всем категориям примерно одинаковые. Около 90% клиентов не имели проблем с возвратом кредита.
Тем не менее лучше остальных выплачивают кредиты вдовцы/вдовы (6,6% имели задолженность), а хуже всех не женатые/не замужние и лица в гражданском браке (9,8% и 9,3% соответственно имели задолженность).

### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
round(data.pivot_table(index='total_income_category', values='debt', aggfunc='mean'),3)

,debt
total_income_category,
A,0.080
B,0.071
C,0.085
D,0.060
E,0.091


### Вывод 3:
Доли по всем категориям примерно одинаковые. Около 90% клиентов не имели проблем с возвратом кредита.
Лучше всего возвращают кредиты клиенты с доходом категории D (30001–50000 руб.), только у 6% была задолженность по выплате кредита. Хуже всего лица с самым маленьким доходом (категория E, до 30000 руб.), 9,1% имели проблемы с выплатой кредита.

### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [45]:
round(data.pivot_table(index='purpose_category', values='debt', aggfunc='mean'),3)

,debt
purpose_category,
операции с автомобилем,0.094
операции с недвижимостью,0.072
получение образования,0.092
проведение свадьбы,0.080


### Вывод 4:
Доли по всем категориям примерно одинаковые. Около 90% клиентов не имели проблем с возвратом кредита.
Лучше всего возвращались кредиты по операциям с недвижимостью и на проведение свадьбы (задолженность была только у 7,2% и 8% соответственно). Кредиты на операции с автомобилем и получение образования выплачивались чуть хуже, ~9% была выплачены с задержкой.

## Общий вывод:

<div style="border:solid blue 1px; padding: 20px"> 
В целом можно сделать вывод, что большинство клиентов банка исправно выплачивают кредиты (порядка 90% клиентов не имели проблем с выплатой задолженности). Есть небольшие отклонения по категориям дохода, целям кредита, семейному положению и наличию детей. Но эти факторы не оказывают сильного влияния на факт выплаты.